## Reading csv file

In [11]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv("Movie_reviews.csv")

# Display the first few rows of the dataframe
df.head()


,review,sentiment
0,This is the kind of film you want to see with ...,positive
1,"First, this was a BRAVE film. I've seen Irreve...",positive
2,I'm not sure why the producers needed to trade...,negative
3,I don't know any idiotic rock'n'roll cliché no...,negative
4,Four stories about the drug trade in Europe be...,positive


## Preprocessing

In [4]:
# Assuming the columns are named 'review' and 'sentiment'
df = df[['review', 'sentiment']]

# Encode the sentiment labels if they are not numerical
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Drop any rows with missing values
df.dropna(inplace=True)

# Split the data into features (X) and labels (y)
X = df['review'].values
y = df['sentiment'].values


## Sentence Transformer

In [6]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
from sentence_transformers import SentenceTransformer

# Load the pretrained Sentence Transformer model
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Calculate embeddings for the reviews
embeddings = sbert_model.encode(X)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Training on NN

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)

# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(384,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 2 classes (positive and negative)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
125/125 [==============================] - 4s 8ms/step - loss: 0.6243 - accuracy: 0.6470 - val_loss: 0.4873 - val_accuracy: 0.7770
Epoch 2/10
125/125 [==============================] - 1s 6ms/step - loss: 0.4909 - accuracy: 0.7690 - val_loss: 0.4428 - val_accuracy: 0.7900
Epoch 3/10
125/125 [==============================] - 1s 5ms/step - loss: 0.4463 - accuracy: 0.7950 - val_loss: 0.4245 - val_accuracy: 0.8070
Epoch 4/10
125/125 [==============================] - 0s 4ms/step - loss: 0.4208 - accuracy: 0.8095 - val_loss: 0.4177 - val_accuracy: 0.8040
Epoch 5/10
125/125 [==============================] - 0s 4ms/step - loss: 0.4006 - accuracy: 0.8220 - val_loss: 0.4241 - val_accuracy: 0.8040
Epoch 6/10
125/125 [==============================] - 0s 4ms/step - loss: 0.3884 - accuracy: 0.8317 - val_loss: 0.4298 - val_accuracy: 0.7900
Epoch 7/10
125/125 [==============================] - 0s 4ms/step - loss: 0.3692 - accuracy: 0.8413 - val_loss: 0.4193 - val_accuracy: 0.8030
Epoch 

In [9]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


32/32 [==============================] - 0s 3ms/step - loss: 0.4271 - accuracy: 0.8110
Test Accuracy: 0.81


## Testing the accuracy of our model

In [16]:
def predict_sentiment(review):
    # Generate embedding for the new review
    review_embedding = sbert_model.encode([review])
    # Predict sentiment
    prediction = model.predict(review_embedding)
    # Convert prediction to class label
    sentiment = np.argmax(prediction, axis=1)[0]
    return 'positive' if sentiment == 1 else 'negative'

# Test the function with a new review
new_review = input("Please enter a review ")
predicted_sentiment = predict_sentiment(new_review)
print(f"Sentiment of the review is: {predicted_sentiment}")

Please enter a review pathetic to say the least
1/1 [==============================] - 0s 17ms/step
Sentiment of the review is: negative
